In [160]:
import pandas as pd
import numpy as np

In [161]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

**Data Cleaning**

In [162]:
df_baseball = pd.read_csv("Hitters.csv")

In [163]:
df_baseball.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


We have missing data in the salary column. There are several routes we could take here. If we really prioritized each data point, and we assumed that the NAs in salary are randomly distributed in the population, we could treat the observations with missing salaries as the test data and fit the model on the other data.

However, in this case, we want to tune our model using GridSearch() so we want to remove all NAs, because cross-validation uses all points from our dataset as training and test data.

In [164]:
df_baseball["Salary"].isna().sum()

59

In [165]:
# I am aware I am dropping 59 observations with dropna().

df_baseball = df_baseball.dropna(subset=["Salary"])
len(df_baseball)

263

**Part I: Different Model Specs**

A. Regression without regularization

In [166]:
X = df_baseball.drop("Salary", axis = 1)
y = df_baseball["Salary"]

In [167]:
def roee_pipeline(model=LinearRegression()):
  """
  Creates a pipeline that dummifies the categorical variables and standardizes the numerical.
  
  Parameters
  ----------
  model: a sci-kit learn function
    model specification
    
  Return
  ------
  fit pipeline
  """

  ct = ColumnTransformer(
    [
      ("dummify", 
      OneHotEncoder(sparse_output = False, handle_unknown='ignore', drop="first"),
      make_column_selector(dtype_include=object)),
      ("standardize", 
      StandardScaler(), 
      make_column_selector(dtype_include=np.number))
    ],
    remainder = "passthrough"
  )

  pipeline = Pipeline(
    [("preprocessing", ct),
    ("model", model)])
    
  return pipeline

In [168]:
def get_coefficients(dataset, model=LinearRegression()):
  """
  Creates a pipeline that dummifies the categorical variables and standardizes the numerical.
  
  Parameters
  ----------
  model: a sci-kit learn function
    model specification
    
  Return
  ------
  coefficients
  """
  X = dataset.drop("Salary", axis = 1)
  y = dataset["Salary"]
  
  ct = ColumnTransformer(
    [
      ("dummify", 
      OneHotEncoder(sparse_output = False, handle_unknown='ignore', drop="first"),
      make_column_selector(dtype_include=object)),
      ("standardize", 
      StandardScaler(), 
      make_column_selector(dtype_include=np.number))
    ],
    remainder = "passthrough"
  )

  pipeline = Pipeline(
    [("preprocessing", ct),
    ("model", model)]).set_output(transform="pandas")

  pipeline_fit = pipeline.fit(X, y)
  coefficients = pipeline.named_steps['model'].coef_
  var_names = ct.fit_transform(X).columns

  df_coef = pd.DataFrame({
    "Predictor": var_names,
    "Coefficient": coefficients
  }).sort_values(by="Coefficient", ascending=False)

  return df_coef

In [169]:
get_coefficients(dataset=df_baseball)

,Predictor,Coefficient
13,standardize__CRuns,480.747135
4,standardize__Hits,337.830479
14,standardize__CRBI,260.689886
8,standardize__Walks,135.073897
11,standardize__CHits,86.687617
16,standardize__PutOuts,78.761296
0,dummify__League_N,62.599423
17,standardize__Assists,53.732490
5,standardize__HmRun,37.853837
12,standardize__CHmRun,-14.181723


In [170]:
df_baseball["CRuns"].std()

331.19857059564885

In [171]:
df_baseball["Hits"].std()

45.12532592258135

In [172]:
df_baseball["CRBI"].std()

323.3676681827309

Career runs, hits in the previous season, and career RBIs are the most significant coefficients by magnitude. 

One standard deviation increase in career runs, 331, is associated with an average increase of $480,000 in salary, holding all other variables constant.

One standard deviation increase in hits from the previous season, 45, is associated with an average increase of $337,000 in salary, holding all other variables constant.

One standard deviation increase in career RBIs, 323, is associated with an average increase of $260,000 in salary, holding all other variables constant.

In [173]:
linear_pipeline = roee_pipeline()
cross_val_score(linear_pipeline, X, y, cv = 5, scoring = "neg_mean_squared_error").mean() * -1

121136.31031816879

B. Ridge Regression

In [174]:
def grid_search_df(dataset, pipeline, model = "ridge"):
  """
  Tunes the model and outputs a Data Frame
  
  Parameters
  ----------
  pipeline: a sci-kit learn Pipeline object
  
  model: str
  can be "linear", "ridge", "lasso", "elastic"

  dataset: pandas Data Frame
    
  Return
  ------
  coefficients
  """
  X = dataset.drop("Salary", axis = 1)
  y = dataset["Salary"]

  if model == "elastic":
    alphas = {
    'model__l1_ratio': [0.2, 0.4, 0.6, 0.8],
    'model__alpha': [0.01, 0.1, 1, 10]
    }
  
  else:
    alphas = {
    'model__alpha': [0.001, 0.01, 0.1, 1, 10]
    }

  gscv = GridSearchCV(pipeline, alphas, cv=5, scoring='neg_mean_squared_error')

  gscv_fitted = gscv.fit(X, y)

  params_df = pd.DataFrame(gscv_fitted.cv_results_["params"])
  results_df = params_df.assign(scores=gscv_fitted.cv_results_["mean_test_score"])

  results_df["scores"] = results_df["scores"] * -1
  return results_df.sort_values(by="scores", ascending=True)

In [175]:
ridge_pipeline = roee_pipeline(model=Ridge())
df_tuned_ridge = grid_search_df(pipeline=ridge_pipeline, model="ridge", dataset=df_baseball)
df_tuned_ridge

,model__alpha,scores
3,1.000,119034.332720
4,10.000,119073.956166
2,0.100,120329.936014
1,0.010,121021.508944
0,0.001,121124.318914


2. The chosen lambda is 1.

In [176]:
ridge_coef = get_coefficients(dataset=df_baseball, model=Ridge(alpha=1))
ridge_coef

,Predictor,Coefficient
13,standardize__CRuns,320.802717
4,standardize__Hits,296.801967
14,standardize__CRBI,160.409497
11,standardize__CHits,126.210585
8,standardize__Walks,124.344012
16,standardize__PutOuts,78.651037
0,dummify__League_N,58.555947
17,standardize__Assists,47.493631
12,standardize__CHmRun,39.055346
5,standardize__HmRun,17.973971


Career runs, hits in the previous season, and career RBIs are the most significant coefficients by magnitude. 

One standard deviation increase in career runs, 331, is associated with an average increase of $320,000 in salary, holding all other variables constant.

One standard deviation increase in hits from the previous season, 45, is associated with an average increase of $296,000 in salary, holding all other variables constant.

One standard deviation increase in career RBIs, 323, is associated with an average increase of $160,000 in salary, holding all other variables constant.

4. With the tuning procedure on this data using Ridge Regression, the lowest MSE (with a lambda value of 0.1) is 119,034.33.

C. Lasso Regression

In [177]:
lasso_pipeline = roee_pipeline(model=Lasso(max_iter=100000))
df_tuned_lasso = grid_search_df(pipeline=lasso_pipeline, model="lasso", dataset=df_baseball)
df_tuned_lasso

,model__alpha,scores
3,1.000,119758.108873
2,0.100,120758.508162
1,0.010,121096.159270
0,0.001,121132.276754
4,10.000,121828.141333


2. The best lambda is 1.

In [178]:
lasso_coef = get_coefficients(dataset=df_baseball, model=Lasso(alpha=1))
lasso_coef

,Predictor,Coefficient
13,standardize__CRuns,375.565519
4,standardize__Hits,304.359509
14,standardize__CRBI,192.610892
8,standardize__Walks,120.695275
16,standardize__PutOuts,78.760366
17,standardize__Assists,41.996680
0,dummify__League_N,35.826072
12,standardize__CHmRun,14.225993
5,standardize__HmRun,11.127022
2,dummify__NewLeague_N,-0.000000


Career runs, hits in the previous season, and career RBIs are the most significant coefficients by magnitude. 

One standard deviation increase in career runs, 331, is associated with an average increase of $375,000 in salary, holding all other variables constant.

One standard deviation increase in hits from the previous season, 45, is associated with an average increase of $304,000 in salary, holding all other variables constant.

One standard deviation increase in career RBIs, 323, is associated with an average increase of $192,000 in salary, holding all other variables constant.

4. With the tuning procedure on this data using Lasso Regression, the lowest MSE (with a lambda value of 0.1) is 119,034.33.

D. Elastic Net

In [179]:
elastic_pipeline = roee_pipeline(model=ElasticNet(max_iter=100000))
df_tuned_elastic = grid_search_df(pipeline=elastic_pipeline, model="elastic", dataset=df_baseball)
df_tuned_elastic.head(n=3)

,model__alpha,model__l1_ratio,scores
7,0.10,0.8,118750.189196
0,0.01,0.2,118836.651842
1,0.01,0.4,118936.657888


2. The best lambda is 0.1 and the best alpha is 0.8 for an MSE of 118,750.

In [180]:
elastic_coef = get_coefficients(dataset=df_baseball, model=ElasticNet(alpha=0.1, l1_ratio=0.8))
elastic_coef

,Predictor,Coefficient
4,standardize__Hits,200.830729
13,standardize__CRuns,163.907498
11,standardize__CHits,110.173426
14,standardize__CRBI,104.504476
8,standardize__Walks,97.341332
16,standardize__PutOuts,76.907372
12,standardize__CHmRun,59.364022
0,dummify__League_N,45.390029
17,standardize__Assists,35.091056
7,standardize__RBI,10.165051


In [181]:
df_baseball["CHits"].std()

648.1996437306398

Career runs, hits in the previous season, and career hits are the most significant coefficients by magnitude in this model.

One standard deviation increase in hits from the previous season, 45, is associated with an average increase of $200,000 in salary, holding all other variables constant.

One standard deviation increase in career runs, 331, is associated with an average increase of $163,000 in salary, holding all other variables constant.

One standard deviation increase in career hits, 648, is associated with an average increase of $110,000 in salary, holding all other variables constant.

4. With the tuning procedure on this data using an Elastic Net, the lowest MSE (with a lambda value of 0.1 and alpha of 0.8) is 118,750.19.

**Part II. Variable Selection**

I have decided that 